In [ ]:
# !pip install -q transformers
!pip install -q bitsandbytes
# !pip install -q datasets
# !pip install -q torch
!pip install -q lm-eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
 

In [ ]:
import torch
from torch.optim import AdamW
from transformers import (AutoModelForCausalLM, Gemma3ForConditionalGeneration, AutoProcessor,
                          AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments)
from tqdm import tqdm
from datasets import load_dataset
from torch.nn import functional as F
from torch.utils.data import DataLoader

In [ ]:
import os
import wandb
from google.colab import userdata

os.environ["HF_TOKEN"] =  userdata.get('HF_TOKEN')
WANDB_API_KEY =  userdata.get('WANDB_API_KEY')

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

WANDB_PROJECT_NAME = "Distil_Gemma_LLama"
wandb.login(key=WANDB_API_KEY)
if len(WANDB_PROJECT_NAME) > 0:
    os.environ["WANDB_PROJECT"] = WANDB_PROJECT_NAME

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohamed-ahmed to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
teacher_model_name = "google/gemma-3-4b-it"
student_model_name = "meta-llama/Llama-3.2-1B"

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(student_model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token

'<|end_of_text|>'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# if torch.cuda.get_device_capability()[0] >= 8:
#     !pip install -qqq flash-attn
#     torch_dtype = torch.bfloat16
#     attn_implementation = "flash_attention_2"
# else:
#     torch_dtype = torch.float16
#     attn_implementation = "eager"

torch_dtype = torch.bfloat16
attn_implementation = "eager"

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_use_double_quant=True,
                                         bnb_4bit_quant_type="nf4",
                                         bnb_4bit_compute_dtype=torch_dtype,
                                         llm_int8_enable_fp32_cpu_offload=False)

In [ ]:
# teacher_model = Gemma3ForConditionalGeneration.from_pretrained(teacher_model_name,
#                                                                quantization_config=quantization_config,
#                                                                device_map="auto",
#                                                                trust_remote_code=True,
#                                                                torch_dtype=torch_dtype,
#                                                                attn_implementation=attn_implementation)

# student_model = AutoModelForCausalLM.from_pretrained(student_model_name,
#                                                      quantization_config=quantization_config,
#                                                      device_map="auto",
#                                                      trust_remote_code=True,
#                                                      attn_implementation=attn_implementation)

teacher_model = Gemma3ForConditionalGeneration.from_pretrained(teacher_model_name,
                                                               quantization_config=quantization_config,
                                                               device_map="auto",
                                                               trust_remote_code=True,
                                                               torch_dtype=torch_dtype,
                                                               attn_implementation=attn_implementation)

student_model = AutoModelForCausalLM.from_pretrained(student_model_name,
                                                     torch_dtype=torch_dtype,
                                                     device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# import torch
# from transformers import AutoProcessor, Gemma3ForConditionalGeneration

# ckpt = "google/gemma-3-4b-it"
# model = Gemma3ForConditionalGeneration.from_pretrained(
#     ckpt,
#     device_map="auto",
#     torch_dtype=torch_dtype)

# processor = AutoProcessor.from_pretrained(ckpt)

# messages = [
#     {
#         "role": "user",
#         "content": [
#             # {"type": "image", "url": "https://huggingface.co/spaces/big-vision/paligemma-hf/resolve/main/examples/password.jpg"},
#             {"type": "text", "text": "What is the password?"}
#         ]
#     }
# ]

# max_length = 512
# inputs = processor.apply_chat_template(
#     messages, add_generation_prompt=True, tokenize=True,
#     return_dict=True, return_tensors="pt", padding_side="right",
#     padding="max_length", truncation=True, max_length=max_length,
# ).to(model.device)


# teacher_labels = inputs["input_ids"].clone()
# teacher_labels[teacher_labels == processor.tokenizer.pad_token_id] = -100
# inputs["labels"] = teacher_labels

# model.eval()
# with torch.no_grad():
#     teacher_outputs = model(
#         input_ids=inputs["input_ids"],
#         attention_mask=inputs["attention_mask"],
#         labels=inputs["labels"],
#         output_hidden_states=True
#     )
# teacher_logits = teacher_outputs.logits

# # input_len = inputs["input_ids"].shape[-1]

# # generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
# # generation = generation[0][input_len:]

# # decoded = processor.decode(generation, skip_special_tokens=True)
# # print(decoded)

# teacher_logits

In [ ]:
dataset = load_dataset("lavita/medical-qa-datasets", "all-processed", split="train")
original_dataset = dataset
dataset = dataset.select(range(1000))

In [ ]:
dataset.column_names

['instruction', 'input', 'output', '__index_level_0__']

In [ ]:
def format_prompt(example):
    return {
        "prompt": f"Instruction: {example['instruction']}\nQuestion{example['input']}\nAnswer: {example['output']}"
    }

formatted_dataset = dataset.map(format_prompt)

In [ ]:
print(formatted_dataset['prompt'][0])

Instruction: If you are a doctor, please answer the medical questions based on the patient's description.
Questionhi. im a home health aide and i have a client with scoliosis in the back and kidney disease. her feet ankles and calves have been swollen for the past 2 weeks. mostly in her feet. she started a patch for pain in her legs 3 weeks ago. she started swelling up almost a week after she started the patch and the pain doctor cut the dose in half and she is still swollen. she has no blood clots in her legs because one of her doctors checked and they said it might be because of her back. what do you think? im concerned because she has been swollen up for to long and theres only so much i can do being her home health aide. we both want to get to the bottom of this swelling she is having
Answer: hi, thanks for contacting chatbot. swelling in the legs and feet can come from many causes, one of them being general circulation or ineffectiveness of the kidneys to rid the body of excess wa

In [ ]:
def process_distillation_data(samples, teacher_processor, student_tokenizer, max_length=128):
    prompts = samples["prompt"]
    if not isinstance(prompts, list):
        prompts = [prompts]

    # Process teacher inputs (text-only)
    teacher_inputs = teacher_processor(
        text=prompts,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        padding_side="right"
    )

    teacher_input_ids = teacher_inputs["input_ids"]
    teacher_attention_mask = teacher_inputs["attention_mask"]

    teacher_labels = teacher_input_ids.clone()
    teacher_labels[teacher_labels == teacher_processor.tokenizer.pad_token_id] = -100

    # Process student inputs (text-only)
    student_inputs = student_tokenizer(
        prompts,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        padding_side="right"
    )

    student_input_ids = student_inputs["input_ids"]
    student_attention_mask = student_inputs["attention_mask"]

    student_labels = student_input_ids.clone()
    student_labels[student_labels == student_tokenizer.pad_token_id] = -100

    return {
        "teacher_input_ids": teacher_input_ids,
        "teacher_labels": teacher_labels,
        "teacher_attention_mask": teacher_attention_mask,
        "student_input_ids": student_input_ids,
        "student_labels": student_labels,
        "student_attention_mask": student_attention_mask
    }


def create_tokenize_function(teacher_processor, student_tokenizer, max_length=128, text_column="prompt"):

    def tokenize_function(examples):
        prompts = examples[text_column]

        # Process teacher inputs
        teacher_inputs = teacher_processor(
            text=prompts,
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt",
            padding_side="right"
        )

        # Process student inputs
        student_inputs = student_tokenizer(
            prompts,
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt",
            padding_side="right"
        )

        return {
            "teacher_input_ids": teacher_inputs["input_ids"].tolist(),
            "teacher_attention_mask": teacher_inputs["attention_mask"].tolist(),
            "teacher_labels": teacher_inputs["input_ids"].tolist(),
            "student_input_ids": student_inputs["input_ids"].tolist(),
            "student_attention_mask": student_inputs["attention_mask"].tolist(),
            "student_labels": student_inputs["input_ids"].tolist(),
        }

    return tokenize_function


def create_distillation_batch(text_prompts, teacher_processor, student_tokenizer, max_length=128):
    if isinstance(text_prompts, str):
        text_prompts = [text_prompts]

    samples = {"prompt": text_prompts}

    return process_distillation_data(samples, teacher_processor, student_tokenizer, max_length)

In [ ]:
teacher_processor = AutoProcessor.from_pretrained(teacher_model_name)
student_tokenizer = AutoTokenizer.from_pretrained(student_model_name)
student_tokenizer.pad_token = student_tokenizer.eos_token
student_tokenizer.pad_token

max_length = 512
tokenize_function = create_tokenize_function(
    teacher_processor=teacher_processor,
    student_tokenizer=student_tokenizer,
    max_length=max_length,
    text_column="prompt"
)

print("Tokenizing dataset .... ")

tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=32,
    remove_columns=formatted_dataset.column_names,
    desc="Processing samples for distillation",
    load_from_cache_file=False
)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Tokenizing dataset .... 


Processing samples for distillation:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.set_format("torch")

In [ ]:
dataloader = DataLoader(
    tokenized_dataset,
    batch_size=3,
    shuffle=True
)

### Optimizer

In [ ]:
lr = 1e-6
optimizer = AdamW(student_model.parameters(), lr=lr)

### Training Loop

In [ ]:
num_epochs = 10
temperature = 2.0
alpha = 1
accumulation_steps = 8

In [ ]:
# batch = next(iter(dataloader))
# batch

In [ ]:
# teacher_model.eval()
# with torch.no_grad():
#     teacher_outputs = teacher_model(
#         input_ids=batch["teacher_input_ids"].to(device),
#         attention_mask=batch["teacher_attention_mask"].to(device),
#         labels=batch["teacher_labels"].to(device),
#         output_hidden_states=True
#     )
# teacher_outputs.logits

In [ ]:
# student_model.eval()
# with torch.no_grad():
#     student_outputs = student_model(
#         input_ids=batch["student_input_ids"].to(device),
#         attention_mask=batch["student_attention_mask"].to(device),
#         labels=batch["student_labels"].to(device),
#         output_hidden_states=True
#     )
# student_outputs.logits
# student_outputs.loss

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from tqdm import tqdm

# # Initialize projection layer if needed (outside the loop for efficiency)
# teacher_proj = None
# student_vocab_size = student_model.get_input_embeddings().weight.size(0)
# teacher_vocab_size = teacher_model.get_input_embeddings().weight.size(0)

# # if teacher_vocab_size != student_vocab_size:
# #     teacher_proj = nn.Linear(
# #         teacher_vocab_size,
# #         student_vocab_size,
# #         dtype=torch.float16,
# #         bias=False
# #     ).to(device)
# #     print(f"Initialized projection layer: {teacher_vocab_size} -> {student_vocab_size}")

# hidden_dim = 8
# if teacher_vocab_size != student_vocab_size:
#     teacher_proj = nn.Sequential(
#         nn.Linear(teacher_vocab_size, hidden_dim, bias=False, dtype=torch.float16),
#         nn.Linear(hidden_dim, student_vocab_size, bias=False, dtype=torch.float16)
#     ).to(device)
#     print(f"Initialized projection layer: {teacher_vocab_size} -> {student_vocab_size}")

# # Knowledge distillation parameters
# alpha = 0.7  # Weight for soft loss (0.7) vs hard loss (0.3)

# for epoch in range(num_epochs):
#     ### 1 - Model Preparation ###
#     student_model.train()
#     teacher_model.eval()  # Ensure teacher is in eval mode

#     total_loss = 0
#     total_hard_loss = 0
#     total_soft_loss = 0

#     for batch_idx, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
#         ### 2 - Data Processing ###
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         labels = batch["labels"].to(device)

#         ### 3 - Teacher Model Inference ###
#         with torch.no_grad():
#             teacher_outputs = teacher_model(
#                 input_ids=input_ids,
#                 attention_mask=attention_mask,
#                 labels=labels,  # Include labels for consistency
#                 output_hidden_states=True
#             )
#             teacher_logits = teacher_outputs.logits

#         ### 4 - Student Model Inference ###
#         student_outputs = student_model(
#             input_ids=input_ids,
#             attention_mask=attention_mask,
#             labels=labels,
#             output_hidden_states=True
#         )
#         student_logits = student_outputs.logits

#         ### 5 - Align Sequence Lengths ###
#         min_seq_len = min(student_logits.size(1), teacher_logits.size(1))
#         student_logits = student_logits[:, :min_seq_len, :]
#         teacher_logits = teacher_logits[:, :min_seq_len, :]

#         ### 6 - Apply Projection if Needed ###
#         if teacher_proj is not None:
#             teacher_logits = teacher_logits.to(device).to(torch.float16)
#             teacher_logits = teacher_proj(teacher_logits)

#         ### 7 - Compute Hard Loss (Standard Cross-Entropy) ###
#         hard_loss = student_outputs.loss

#         ### 8 - Compute Soft Loss (Knowledge Distillation) ###
#         # Apply temperature scaling
#         student_log_probs = F.log_softmax(student_logits / temperature, dim=-1)
#         teacher_probs = F.softmax(teacher_logits / temperature, dim=-1)

#         # KL Divergence loss
#         soft_loss = F.kl_div(
#             student_log_probs,
#             teacher_probs,
#             reduction='batchmean'
#         ) * (temperature ** 2)

#         ### 9 - Combine Losses ###
#         total_batch_loss = alpha * soft_loss + (1 - alpha) * hard_loss

#         # Scale for gradient accumulation
#         total_batch_loss = total_batch_loss / accumulation_steps

#         ### 10 - Backward Pass ###
#         total_batch_loss.backward()

#         ### 11 - Optimization with Gradient Accumulation ###
#         if ((batch_idx + 1) % accumulation_steps == 0) or (batch_idx + 1 == len(dataloader)):
#             optimizer.step()
#             optimizer.zero_grad()

#         ### 12 - Loss Tracking ###
#         # Scale losses back up for reporting
#         batch_total_loss = total_batch_loss.item() * accumulation_steps
#         batch_hard_loss = hard_loss.item()
#         batch_soft_loss = soft_loss.item()

#         total_loss += batch_total_loss
#         total_hard_loss += batch_hard_loss
#         total_soft_loss += batch_soft_loss

#         if (batch_idx + 1) % 100 == 0:
#             print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}")
#             print(f"  Total Loss: {batch_total_loss:.4f}")
#             print(f"  Hard Loss: {batch_hard_loss:.4f}")
#             print(f"  Soft Loss: {batch_soft_loss:.4f}")

#     ### 13 - Epoch-Level Reporting ###
#     avg_total_loss = total_loss / len(dataloader)
#     avg_hard_loss = total_hard_loss / len(dataloader)
#     avg_soft_loss = total_soft_loss / len(dataloader)

#     print(f"\nEpoch {epoch+1}/{num_epochs} Summary:")
#     print(f"  Average Total Loss: {avg_total_loss:.4f}")
#     print(f"  Average Hard Loss: {avg_hard_loss:.4f}")
#     print(f"  Average Soft Loss: {avg_soft_loss:.4f}")
#     print("-" * 50)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

teacher_proj = None
student_vocab_size = student_model.get_input_embeddings().weight.size(0)
teacher_vocab_size = teacher_model.get_input_embeddings().weight.size(0)

hidden_dim = 4096
if teacher_vocab_size != student_vocab_size:
    teacher_proj = nn.Sequential(
        nn.Linear(teacher_vocab_size, hidden_dim, bias=False, dtype=torch.float16),
        nn.ReLU(),
        nn.Linear(hidden_dim, student_vocab_size, bias=False, dtype=torch.float16)
    ).to(device)

    with torch.no_grad():
        nn.init.xavier_uniform_(teacher_proj[0].weight)
        nn.init.xavier_uniform_(teacher_proj[2].weight)
        teacher_proj[0].weight *= 0.1
        teacher_proj[2].weight *= 0.1

    print(f"Initialized projection layer: {teacher_vocab_size} -> {student_vocab_size}")

# Knowledge distillation parameters
alpha = 0.7  # Weight for soft loss (0.7) vs hard loss (0.3)
temperature = 4.0  # temperature parameter
accumulation_steps = 1

print("Checking student model for NaN parameters...")
nan_params = []
for name, param in student_model.named_parameters():
    if torch.isnan(param).any() or torch.isinf(param).any():
        nan_params.append(name)
        print(f"NaN/Inf detected in parameter: {name}")

if nan_params:
    print("CRITICAL: Model has NaN parameters! Reinitialize the model or restore from checkpoint.")
    # Option 1: Reinitialize problematic layers
    for name, module in student_model.named_modules():
        if any(pname.startswith(name) for pname in nan_params):
            if hasattr(module, 'weight') and module.weight is not None:
                nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if hasattr(module, 'bias') and module.bias is not None:
                nn.init.zeros_(module.bias)
    print("Reinitialized problematic parameters")

print(f"Learning rate: {optimizer.param_groups[0]['lr']}")
if optimizer.param_groups[0]['lr'] == 0 or optimizer.param_groups[0]['lr'] > 1e-4:
    print("WARNING: Learning rate might be problematic!")
    # Reset to a very safe learning rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = 1e-7  # Even more conservative

# Check for any existing gradient accumulation
optimizer.zero_grad()

# Enable anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)

for epoch in range(num_epochs):
    ### 1 - Model Preparation ###
    student_model.train()
    teacher_model.eval()

    total_loss = 0
    total_hard_loss = 0
    total_soft_loss = 0
    num_valid_batches = 0

    for batch_idx, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        ### 2 - Data Processing and Validation ###
        try:
            teacher_input_ids = batch["teacher_input_ids"].to(device)
            teacher_attention_mask = batch["teacher_attention_mask"].to(device)
            teacher_labels = batch["teacher_labels"].to(device)

            student_input_ids = batch["student_input_ids"].to(device)
            student_attention_mask = batch["student_attention_mask"].to(device)
            student_labels = batch["student_labels"].to(device)

            # CRITICAL: Check for invalid labels
            if torch.isnan(student_labels).any() or torch.isinf(student_labels).any():
                print(f"Invalid labels detected at batch {batch_idx}")
                continue

            # Check if all labels are -100 (ignore_index)
            valid_labels = (student_labels != -100).sum()
            if valid_labels == 0:
                print(f"No valid labels in batch {batch_idx}")
                continue

            print(f"Batch {batch_idx}: Valid labels = {valid_labels.item()}")

        except Exception as e:
            print(f"Error in data processing at batch {batch_idx}: {e}")
            continue

        ### 3 - Teacher Model Inference ###
        try:
            with torch.no_grad():
                teacher_outputs = teacher_model(
                    input_ids=teacher_input_ids,
                    attention_mask=teacher_attention_mask,
                    labels=teacher_labels,
                    output_hidden_states=True
                )
                teacher_logits = teacher_outputs.logits

                # Check teacher outputs for NaN
                if torch.isnan(teacher_logits).any():
                    print(f"NaN in teacher logits at batch {batch_idx}")
                    continue

        except Exception as e:
            print(f"Error in teacher inference at batch {batch_idx}: {e}")
            continue

        ### 4 - Student Model Inference ###
        try:
            # CRITICAL: Check for NaN in inputs before forward pass
            if torch.isnan(student_input_ids.float()).any():
                print(f"NaN in student input_ids at batch {batch_idx}")
                continue
            if torch.isnan(student_attention_mask.float()).any():
                print(f"NaN in student attention_mask at batch {batch_idx}")
                continue

            student_outputs = student_model(
                input_ids=student_input_ids,
                attention_mask=student_attention_mask,
                labels=student_labels,
                output_hidden_states=True
            )
            student_logits = student_outputs.logits

            # Check student outputs for NaN/Inf
            if torch.isnan(student_logits).any() or torch.isinf(student_logits).any():
                print(f"NaN/Inf in student logits at batch {batch_idx}")
                print(f"  Logits stats: min={student_logits.min():.4f}, max={student_logits.max():.4f}")

                # Check which parameters have NaN after this forward pass
                for name, param in student_model.named_parameters():
                    if torch.isnan(param).any():
                        print(f"  NaN in parameter {name} after forward pass")

                # EMERGENCY: Reinitialize the problematic parameters
                print("EMERGENCY: Reinitializing model parameters due to NaN")
                for name, module in student_model.named_modules():
                    if hasattr(module, 'weight') and module.weight is not None:
                        if torch.isnan(module.weight).any():
                            nn.init.normal_(module.weight, mean=0.0, std=0.02)
                            print(f"  Reinitialized {name}.weight")
                    if hasattr(module, 'bias') and module.bias is not None:
                        if torch.isnan(module.bias).any():
                            nn.init.zeros_(module.bias)
                            print(f"  Reinitialized {name}.bias")

                # Skip this batch
                continue

        except Exception as e:
            print(f"Error in student inference at batch {batch_idx}: {e}")
            continue

        ### 5 - Align Sequence Lengths ###
        min_seq_len = min(student_logits.size(1), teacher_logits.size(1))
        student_logits = student_logits[:, :min_seq_len, :]
        teacher_logits = teacher_logits[:, :min_seq_len, :]

        ### 6 - Apply Projection if Needed ###
        if teacher_proj is not None:
            try:
                teacher_logits = teacher_proj(teacher_logits.to(torch.float16))
                if torch.isnan(teacher_logits).any():
                    print(f"NaN after projection at batch {batch_idx}")
                    continue
            except Exception as e:
                print(f"Error in projection at batch {batch_idx}: {e}")
                continue

        ### 7 - Compute Hard Loss (FIXED) ###
        # CRITICAL: Manually compute cross-entropy to debug
        try:
            # Get the aligned labels
            student_labels_aligned = student_labels[:, :min_seq_len]

            # Create mask for valid positions
            mask = (student_labels_aligned != -100)

            if mask.sum() == 0:
                print(f"No valid positions for loss computation at batch {batch_idx}")
                continue

            # Flatten for cross-entropy computation
            flat_logits = student_logits.view(-1, student_logits.size(-1))
            flat_labels = student_labels_aligned.view(-1)

            # Compute cross-entropy manually with proper handling
            hard_loss = F.cross_entropy(
                flat_logits,
                flat_labels,
                ignore_index=-100,
                reduction='mean'
            )

            print(f"Batch {batch_idx} - hard_loss: {hard_loss}")

            # Check for NaN in hard loss
            if torch.isnan(hard_loss) or torch.isinf(hard_loss):
                print(f"Invalid hard loss at batch {batch_idx}: {hard_loss}")
                # Debug information
                print(f"  Logits range: [{flat_logits.min():.4f}, {flat_logits.max():.4f}]")
                print(f"  Labels range: [{flat_labels.min()}, {flat_labels.max()}]")
                print(f"  Valid labels: {mask.sum()}")
                continue

        except Exception as e:
            print(f"Error computing hard loss at batch {batch_idx}: {e}")
            continue

        ### 8 - Compute Soft Loss (Knowledge Distillation) ###
        try:
            # Only compute on non-padded positions - use student_labels for mask
            mask = (student_labels_aligned != -100).float()

            if mask.sum() == 0:
                soft_loss = torch.tensor(0.0, device=device)
            else:
                # Apply temperature scaling with stability checks
                student_logits_temp = student_logits / temperature
                teacher_logits_temp = teacher_logits / temperature

                # Check for extreme values before softmax
                if torch.abs(student_logits_temp).max() > 50 or torch.abs(teacher_logits_temp).max() > 50:
                    print(f"Extreme logit values at batch {batch_idx}")
                    continue

                student_log_probs = F.log_softmax(student_logits_temp, dim=-1)
                teacher_probs = F.softmax(teacher_logits_temp, dim=-1)

                # Check for NaN in probabilities
                if torch.isnan(student_log_probs).any() or torch.isnan(teacher_probs).any():
                    print(f"NaN detected in probabilities at batch {batch_idx}")
                    continue

                # Compute KL divergence only on valid positions
                kl_loss = F.kl_div(
                    student_log_probs,
                    teacher_probs,
                    reduction='none',
                    log_target=False
                )

                # Apply mask and compute mean
                kl_per_token = kl_loss.sum(-1)  # Sum over vocab dimension
                masked_kl = kl_per_token * mask  # Apply mask
                soft_loss = masked_kl.sum() / mask.sum()  # Average over valid tokens
                soft_loss = soft_loss * (temperature ** 2)  # Scale by temperature squared

            print(f"Batch {batch_idx} - soft_loss: {soft_loss}")

            # Check for NaN in soft loss
            if torch.isnan(soft_loss) or torch.isinf(soft_loss):
                print(f"Invalid soft loss at batch {batch_idx}: {soft_loss}")
                continue

        except Exception as e:
            print(f"Error computing soft loss at batch {batch_idx}: {e}")
            continue

        ### 9 - Combine Losses ###
        total_batch_loss = alpha * soft_loss + (1 - alpha) * hard_loss

        print(f"Batch {batch_idx} - total_batch_loss: {total_batch_loss}")

        # Final NaN check
        if torch.isnan(total_batch_loss) or torch.isinf(total_batch_loss):
            print(f"Invalid total loss at batch {batch_idx}: {total_batch_loss}")
            continue

        # Scale for gradient accumulation
        total_batch_loss = total_batch_loss / accumulation_steps

        ### 10 - Backward Pass with Error Handling ###
        try:
            total_batch_loss.backward()

            # Check for NaN gradients
            has_nan_grad = False
            for name, param in student_model.named_parameters():
                if param.grad is not None and torch.isnan(param.grad).any():
                    print(f"NaN gradient in {name}")
                    has_nan_grad = True
                    break

            if has_nan_grad:
                optimizer.zero_grad()
                continue

            # Gradient clipping to prevent explosion
            grad_norm = torch.nn.utils.clip_grad_norm_(student_model.parameters(), max_norm=1.0)
            if teacher_proj is not None:
                torch.nn.utils.clip_grad_norm_(teacher_proj.parameters(), max_norm=1.0)

            print(f"Batch {batch_idx} - gradient norm: {grad_norm:.4f}")

        except Exception as e:
            print(f"Error in backward pass at batch {batch_idx}: {e}")
            optimizer.zero_grad()
            continue

        ### 11 - Optimization with Gradient Accumulation ###
        if ((batch_idx + 1) % accumulation_steps == 0) or (batch_idx + 1 == len(dataloader)):
            try:
                optimizer.step()
                optimizer.zero_grad()
            except Exception as e:
                print(f"Error in optimizer step at batch {batch_idx}: {e}")
                optimizer.zero_grad()
                continue

        ### 12 - Loss Tracking ###
        # Scale losses back up for reporting
        batch_total_loss = total_batch_loss.item() * accumulation_steps
        batch_hard_loss = hard_loss.item()
        batch_soft_loss = soft_loss.item()

        # Only track valid (non-NaN) losses
        if not (torch.isnan(torch.tensor(batch_total_loss)) or
                torch.isnan(torch.tensor(batch_hard_loss)) or
                torch.isnan(torch.tensor(batch_soft_loss))):
            total_loss += batch_total_loss
            total_hard_loss += batch_hard_loss
            total_soft_loss += batch_soft_loss
            num_valid_batches += 1

        if (batch_idx + 1) % 10 == 0:  # More frequent reporting for debugging
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}")
            print(f"  Total Loss: {batch_total_loss:.4f}")
            print(f"  Hard Loss: {batch_hard_loss:.4f}")
            print(f"  Soft Loss: {batch_soft_loss:.4f}")

    ### 13 - Epoch-Level Reporting ###
    if num_valid_batches > 0:
        avg_total_loss = total_loss / num_valid_batches
        avg_hard_loss = total_hard_loss / num_valid_batches
        avg_soft_loss = total_soft_loss / num_valid_batches
    else:
        avg_total_loss = avg_hard_loss = avg_soft_loss = float('nan')

    print(f"\nEpoch {epoch+1}/{num_epochs} Summary:")
    print(f"  Average Total Loss: {avg_total_loss:.4f}")
    print(f"  Average Hard Loss: {avg_hard_loss:.4f}")
    print(f"  Average Soft Loss: {avg_soft_loss:.4f}")
    print(f"  Valid Batches: {num_valid_batches}/{len(dataloader)}")
    print("-" * 50)

Initialized projection layer: 262208 -> 128256
Checking student model for NaN parameters...
Learning rate: 1e-06


  0%|          | 0/334 [00:00<?, ?it/s]

Batch 0: Valid labels = 1536
Batch 0 - hard_loss: 11.125
Batch 0 - soft_loss: 3.087339401245117
Batch 0 - total_batch_loss: 5.504887580871582


  0%|          | 1/334 [00:02<12:18,  2.22s/it]

Batch 0 - gradient norm: 209.0000
Batch 1: Valid labels = 1536
Batch 1 - hard_loss: 11.75
Batch 1 - soft_loss: 3.3279716968536377
Batch 1 - total_batch_loss: 5.860830307006836


...

Batch 179 - gradient norm: 117.5000
Epoch 1/10, Batch 180
  Total Loss: 3.1284
  Hard Loss: 6.0000
  Soft Loss: 1.9021
Batch 180: Valid labels = 1536
Batch 180 - hard_loss: 2.625
Batch 180 - soft_loss: 1.0915838479995728
Batch 180 - total_batch_loss: 1.553171157836914


 54%|█████▍    | 181/334 [04:28<03:45,  1.47s/it]

Batch 180 - gradient norm: 36.2500
Batch 181: Valid labels = 1536
Batch 181 - hard_loss: 2.1875
Batch 181 - soft_loss: 1.0630394220352173
Batch 181 - total_batch_loss: 1.4003775119781494


 54%|█████▍    | 182/334 [04:29<03:43,  1.47s/it]

Batch 181 - gradient norm: 34.5000
Batch 182: Valid labels = 1536


### Store the Model

In [ ]:
distil_student_model_name = "pruned_distil_Gemma_3_Llama-3.2-1B"

In [ ]:
student_model.save_pretrained(distil_student_model_name)
tokenizer.save_pretrained(distil_student_model_name)

('pruned_distil_Gemma_3_Llama-3.2-1B/tokenizer_config.json',
 'pruned_distil_Gemma_3_Llama-3.2-1B/special_tokens_map.json',
 'pruned_distil_Gemma_3_Llama-3.2-1B/tokenizer.json')

In [ ]:
student_model.push_to_hub(distil_student_model_name,
                          private=False,
                          use_temp_dir=False)
tokenizer.push_to_hub(distil_student_model_name,
                      private=False,
                      use_temp_dir=False)

  0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/TachyHealthResearch/pruned_distil_Gemma_3_Llama-3.2-1B/commit/72ee56f189fa27337a782d42584995f36cea1686', commit_message='Upload tokenizer', commit_description='', oid='72ee56f189fa27337a782d42584995f36cea1686', pr_url=None, repo_url=RepoUrl('https://huggingface.co/TachyHealthResearch/pruned_distil_Gemma_3_Llama-3.2-1B', endpoint='https://huggingface.co', repo_type='model', repo_id='TachyHealthResearch/pruned_distil_Gemma_3_Llama-3.2-1B'), pr_revision=None, pr_num=None)

### Evaluating the model

In [ ]:
from lm_eval import evaluator, tasks, models

def evaluate_hf_model(model_name, tasks=['arc_easy'], num_fewshot=0):
    model_args = f"pretrained={model_name},device=cuda"
    tasks = tasks

    results = evaluator.simple_evaluate(
      model="hf",
      model_args=model_args,
      tasks=tasks,
      num_fewshot=0,
      limit=None,
      bootstrap_iters=10
    )

    metrics = results.get('results', {})
    return metrics

In [ ]:
tasks = ['lambada']

metrics_pruned_kd = evaluate_hf_model("TachyHealthResearch/pruned_distil_Gemma_3_Llama-3.2-1B", tasks=tasks)

metrics_pruned_kd

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

...

bootstrapping for stddev: perplexity


100%|██████████| 1/1 [00:00<00:00, 91.61it/s]


bootstrapping for stddev: perplexity


100%|██████████| 1/1 [00:00<00:00, 110.81it/s]


{'lambada_openai': {'alias': 'lambada_openai',
  'perplexity,none': 5.703459960872111,
  'perplexity_stderr,none': 0.19244903316877648,
  'acc,none': 0.6295361925092179,
  'acc_stderr,none': 0.006728144610304273},
 'lambada_standard': {'alias': 'lambada_standard',
  'perplexity,none': 8.854622024627842,
  'perplexity_stderr,none': 0.38517807743498733,
  'acc,none': 0.5350281389481856,
  'acc_stderr,none': 0.006948862533178278}}